In [ ]:
!pip install torch pandas numpy scikit-learn
!pip install transformers datasets nltk
!pip install pdfplumber rouge_score arxiv
!pip install bs4 requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Import necessary libraries
import torch
import pandas as pd
import random
import numpy as np
from transformers import (
    AutoTokenizer, BartForConditionalGeneration, BartTokenizer,
    AutoModelForSeq2SeqLM,
    Trainer,
    TrainingArguments,
    TrainerCallback,
    DataCollatorForSeq2Seq
)
from datasets import Dataset, DatasetDict
import pdfplumber
import re
from sklearn.model_selection import train_test_split
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import nltk
from rouge_score import rouge_scorer
import arxiv
from bs4 import BeautifulSoup
import requests
import json
from tqdm import tqdm
import matplotlib.pyplot as plt

# Download required NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Initialize the model and tokenizer
model_name = "facebook/bart-base"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [ ]:
import pandas as pd
import json

# Load the JSON file
with open('/content/combined.json', 'r') as f:
    data = json.load(f)

# Convert JSON data to DataFrame
if isinstance(data, list):  # JSON is a list of dictionaries
    df = pd.DataFrame(data)
elif isinstance(data, dict):  # JSON is a dictionary
    df = pd.DataFrame([data])  # Convert dict to DataFrame with one row
else:
    raise ValueError("The JSON file contains an unsupported structure.")

# Display the DataFrame
print(df)

# Optionally, save the DataFrame to a CSV file
df.to_csv('data.csv', index=False)


# Generate synthetic dataset
print("Generating synthetic dataset...")
synthetic_data = df

                  topic                                               text  \
0          Biodiversity  Biodiversity, or biological diversity, is the ...   
1        Climate Change  Climate change, driven largely by human activi...   
2             Pollution  Pollution, the contamination of the environmen...   
3    Arctic ice melting  The melting of Arctic sea ice is one of the mo...   
4     Rising sea levels  Rising sea levels represent a significant and ...   
..                  ...                                                ...   
323  urban environments  Urban environments, centers of human activity ...   
324       coastal areas  Coastal areas, the dynamic zones where land me...   
325    mountain regions  Mountain regions, characterized by their high ...   
326    wetland habitats  Wetland habitats, areas where water covers the...   
327   marine ecosystems  Marine ecosystems, encompassing the vast ocean...   

                                               summary  
0    B

In [ ]:
def tokenize_bart(example):
    # tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    inputs = tokenizer(example['text'], max_length=1024, padding='max_length', truncation=True, return_tensors="pt")
    targets = tokenizer(example['summary'], max_length=512, padding='max_length', truncation=True, return_tensors="pt")
    example['input_ids'] = inputs['input_ids'][0]
    example['attention_mask'] = inputs['attention_mask'][0]
    example['labels'] = targets['input_ids'][0]
    return example

In [ ]:
from sklearn.model_selection import train_test_split

# Splitting the synthetic dataset
train_data, test_data = train_test_split(synthetic_data, test_size=0.2, random_state=42)

# Assign the train dataset as bart_train
bart_train = train_data

# Verify the split
print(f"Training set size: {len(bart_train)}")
print(f"Test set size: {len(test_data)}")

Training set size: 262
Test set size: 66


In [ ]:
from datasets import Dataset

# Convert train and test dataframes to Hugging Face Dataset
bart_train = Dataset.from_pandas(train_data)
bart_test = Dataset.from_pandas(test_data)

# Tokenize the datasets
bart_train = bart_train.map(tokenize_bart)
bart_test = bart_test.map(tokenize_bart)

Map:   0%|          | 0/262 [00:00<?, ? examples/s]

Map:   0%|          | 0/66 [00:00<?, ? examples/s]

In [ ]:
# Define fine-tuning function
def fine_tune_model(model, train_dataset, test_dataset, model_name):
    training_args = TrainingArguments(
        output_dir=f'./results_{model_name}',
        evaluation_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        num_train_epochs=3,  # Adjust as needed
        weight_decay=0.01,
        logging_dir=f'./logs_{model_name}',
        save_total_limit=2,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
    )

    trainer.train()
    return trainer

# Fine-tune the BART model
bart_trainer = fine_tune_model(model, bart_train, bart_test, 'bart-base')

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,0.199065
2,No log,0.153006
3,No log,0.142625


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00


In [ ]:
import torch
import evaluate

# Load the ROUGE metric
rouge = evaluate.load("rouge")

# Evaluation Function with ROUGE Score
def evaluate_model(trainer, test_dataset, tokenizer):
    predictions = []
    references = []

    # Ensure model is on the correct device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    trainer.model.to(device)

    for data in test_dataset:
        # Move input_ids to the same device as the model
        input_ids = torch.tensor(data["input_ids"]).unsqueeze(0).to(device)

        # Generate summary
        summary_ids = trainer.model.generate(
            input_ids,
            max_length=512,
            min_length=10,
            num_beams=4,
            early_stopping=True
        )
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        # Append prediction and reference
        predictions.append(summary)
        references.append(data["summary"])

    # Calculate ROUGE scores
    results = rouge.compute(predictions=predictions, references=references)
    return results

# Example usage
# Evaluate using bart_trainer (which is the trainer object created for fine-tuning BART)
results = evaluate_model(bart_trainer, bart_test, tokenizer)
print("ROUGE Scores:")
print(results)


ROUGE Scores:
{'rouge1': 0.6024538541512916, 'rouge2': 0.38763024854054273, 'rougeL': 0.535507280206067, 'rougeLsum': 0.534617066626393}


In [ ]:
def evaluate_model(trainer, test_dataset, tokenizer):
    predictions = []
    references = []

    # Ensure model is on the correct device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    trainer.model.to(device)

    for data in test_dataset:
        # Move input_ids to the same device as the model
        input_ids = torch.tensor(data["input_ids"]).unsqueeze(0).to(device)

        # Generate summary
        summary_ids = trainer.model.generate(input_ids, max_length=512, min_length=10)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        # Append prediction and reference
        predictions.append(summary)
        references.append(data["summary"])

    # Calculate ROUGE scores
    results = rouge.compute(predictions=predictions, references=references)
    return results

In [ ]:
save_directory = "/content/Momo_bart_model"

# Save BART model and tokenizer
model.save_pretrained(save_directory)
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
tokenizer.save_pretrained(save_directory)

('/content/Momo_bart_model/tokenizer_config.json',
 '/content/Momo_bart_model/special_tokens_map.json',
 '/content/Momo_bart_model/vocab.json',
 '/content/Momo_bart_model/merges.txt',
 '/content/Momo_bart_model/added_tokens.json')

In [ ]:
import shutil

# Zip the directory
shutil.make_archive(save_directory, 'zip', save_directory)

# Confirm the zip file exists
print("Zip file created at:", f"{save_directory}.zip")

Zip file created at: /content/Momo_bart_model.zip


In [ ]:
from google.colab import files

# Download the zip file
files.download(f"{save_directory}.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>